In [1]:
import sys

sys.path.insert(0, '../03_Remake_with_TDD/algorithms')
from ht_time import *

sys.path.insert(0, '../03_Remake_with_TDD/postgresql')
from ht_player_postgresql import *
from ht_match_postgresql import *

/home/rwoo/02_WorkSpace/04_Hattrick/Hattrick-Study/99_Utility/ht-venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
ht_time = HattrickTime()
folder = ht_time.findLastSunOrWed()
# folder = '2018/04/08'
print("folder =", folder)

folder = 2018/11/21


In [3]:
import re
from bs4 import BeautifulSoup

In [6]:
import sys
sys.path.insert(0, '../97_Parse/')
from parseMatch import *

sys.path.insert(0, '../98_Crawler/01_DualMonitor/')
from DualMonitor import getIsDualMonitor

sys.path.insert(0, '../98_Crawler/02_WebBrowser/')
from FirefoxTab import clickTabOfFireFox
from crawMatch import *

sys.path.insert(0, '../98_Crawler/03_OnScreen/')
from Empty import clickEmpty
from MoveScroll import moveScroll
from MoveFindMoveClick import moveByImage, moveByImageWithModify

sys.path.insert(0, '../98_Crawler/04_Login/')
from HattrickLogInOut import clickHattrickLogin, clickHattrickLogout

In [5]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

clickTabOfFireFox(4, isDualMonitor)

clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(510, 305, isDualMonitor)

# clickHattrickLogout(1270, 370, isDualMonitor)

use 2 monitor = 3840 x 1080 isDualMonitor= True


KeyboardInterrupt: 

In [8]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick HeavyDefence Tab
clickTabOfFireFox(4, isDualMonitor)

# Hattrick Login by Facebook
clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(510, 305, isDualMonitor)

# Hattrick MyClub
moveScroll(20)
moveByImage(517, 336, 79, 44, isDualMonitor, 'screen-MyClub.png')

# Hattrick Match
clickEmpty(700, isDualMonitor)
moveScroll(20)
moveByImage(380, 698, 84, 100, isDualMonitor, 'screen-Match.png')

# Hattrick LastMatch
clickEmpty(831, isDualMonitor)
moveScroll(20)
# moveByImageWithModify(1040, 831, 15, -15, 58, 48, isDualMonitor, 'screen-LastMatch.png')
moveAndClick(1095, 845, isDualMonitor)
time.sleep(5)

# FirefoxFile
moveAndClick(90, 15, isDualMonitor)

# # SavePageAs
# time.sleep(3)
# moveAndClick(91, 130, isDualMonitor)

# # Type match
# time.sleep(3)
# typeFileName()

# # rwoo folder
# moveAndClick(570, 460, isDualMonitor)

# # 02_workspace folder
# moveAndClicks(720, 440, isDualMonitor, 2)

# # 04_Hattrick folder
# moveAndClicks(720, 500, isDualMonitor, 2)

# # HattrickStudy
# moveAndClicks(720, 415, isDualMonitor, 2)

# # 01_Heavy2Defence
# moveAndClicks(720, 440, isDualMonitor, 2)

# # 2018
# moveAndClicks(720, 440, isDualMonitor, 2)

# # Last Month
# clickLastMonthOrDay()

# # Last Day
# clickLastMonthOrDay()

# # Save
# moveByImage(1280, 847, 91, 41, isDualMonitor, 'screen-Save.png')

# # Hattrick Logout by Facebook
# clickEmpty(336, isDualMonitor)
# moveScroll(20)
# clickHattrickLogout(1270, 370, isDualMonitor)

# # Jupyter Tab
# clickTabOfFireFox(7, isDualMonitor)

use 2 monitor = 3840 x 1080 isDualMonitor= True
screen-MyClub.png location on Screen ( 517 336 79 44 )
screen-Match.png location on Screen ( 380 698 84 100 )


In [9]:
from ht_file import *
HattrickFile = HattrickFile()

In [10]:
HattrickFile.check_and_delete_folder(folder + '/match_files')

2018/11/21/match_files		 : is exitsted
2018/11/21/match_files		 : will be removed
2018/11/21/match_files		 : is not exitsed


In [11]:
from ht_match import *

In [12]:
HattrickMatch = HattrickMatch()

In [13]:
# ht_player_table_name = 'player'
ht_player_table_name = 'player_tmp'

In [14]:
match_dict_list = HattrickMatch.findMatchList(folder + '/match.html', 'mydatabase2', ht_player_table_name)

Away


In [15]:
match_str_list = HattrickMatch.matchDictListToMatchStrList(match_dict_list)

print("\ncat match_str_List\n")
for str_line in match_str_list:
    print(str_line)


cat match_str_List

po,num,rt,sMin,eMin
KP,1,3.5,0,90
WB,72,2.5,0,90
WB,75,3,0,90
CD,64,5.5,0,90
CD,69,3.5,0,90
CD,68,5,0,90
W,43,5.5,0,90
W,63,6.5,0,16
W,7,4,16,90
IM,31,7,0,90
IM,12,5.5,0,65
IM,10,6.0,65,90
FW,16,6,0,90


In [16]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

2018/11/21/match.txt	 : is not existed


In [17]:
for line in match_str_list:
    with open (folder + '/match.txt', 'a') as f:
        f.write(line + "\n")

In [18]:
import psycopg2

In [19]:
conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
conn

<connection object at 0x7ff7e2d5b768; dsn: 'user=myuser password=xxx dbname=mydatabase2 host=localhost port=65432', closed: 0>

In [20]:
hattrickMatchPostgresql = HattrickMatchPostgreSQL()
hattrickPlayerPostgresql = HattrickPlayerPostgreSQL()

In [21]:
hattrickMatchPostgresql.select_count_of_match(conn)

1089

In [22]:
import csv
with open(folder + '/match.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match( '^[0-9]{1,2}', row[1] ) ) :
            hattrickMatchPostgresql.insert_to_match(conn, folder.replace('/', '-'), row)

In [23]:
hattrickMatchPostgresql.select_count_of_match(conn)

1102

In [24]:
hattrickMatchPostgresql.update_rt_of_match(conn, ht_player_table_name)

In [25]:
tupleList = hattrickMatchPostgresql.getMatchByDate(folder, conn)
hattrickMatchPostgresql.printMatchByDate(tupleList)

KP	1	3.5	0	90
WB	72	2.5	0	90
WB	75	3.0	0	90
CD	69	3.0	0	90
CD	64	5.0	0	90
CD	68	4.5	0	90
W	63	6.5	0	16
W	7	3.5	16	90
W	43	5.5	0	90
IM	12	5.5	0	65
IM	10	6.0	65	90
IM	31	6.5	0	90
FW	16	6.0	0	90


In [26]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

2018/11/21/match.txt	 : will be removed
2018/11/21/match.txt	 : is not existed


In [27]:
with open ( folder + '/match.txt', 'a' ) as f:
    f.write("po,num,rt,sMin,eMin\n")
    f.write( '\n'.join('%s,%s,%s,%s,%s' % myTuple for myTuple in tupleList) + '\n' )

In [28]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'KP', [1,74])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [1,74])

date num min/rt |    01   |    74   |
----------------|---------|---------|
2018-10-24      |  90/4.0 |         |
2018-10-28      |         |         |
2018-10-31      |  90/4.5 |         |
2018-11-04      |         |         |
2018-11-07      |  90/4.0 |         |
2018-11-11      |         |  90/3.0 |
2018-11-14      |  90/3.5 |         |
2018-11-18      |         |  90/3.0 |
2018-11-21      |  90/3.5 |         |


In [29]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'WB', [67,71,72,75])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [67,71,72,75])
tupleList = hattrickPlayerPostgresql.select(conn, ht_player_table_name, folder, [67,71,72,75], 'wb')
print('Num', 'b_p', 'b_p_v', 'wbd_p', 'wb_p', 'wbtm_p', 'wbo_p', sep='\t')
hattrickPlayerPostgresql.print(tupleList, sep='\t')

date num min/rt |    67   |    71   |    72   |    75   |
----------------|---------|---------|---------|---------|
2018-10-24      |  90/4.0 |         |  90/3.5 |         |
2018-10-28      |         |  90/2.5 |         |  90/2.5 |
2018-10-31      |  90/4.0 |         |  90/3.5 |         |
2018-11-04      |         |  90/3.0 |         |  90/2.5 |
2018-11-07      |  90/4.0 |         |  90/3.0 |         |
2018-11-11      |  52/3.5 |  90/3.0 |         |  38/3.0 |
2018-11-14      |  90/4.0 |         |  90/3.0 |         |
2018-11-18      |  90/3.5 |  90/3.0 |         |         |
2018-11-21      |         |         |  90/2.5 |  90/3.0 |
Num	b_p	b_p_v	wbd_p	wb_p	wbtm_p	wbo_p
67	WBd	6.45	6.45	5.95	6.04	5.45
71	Cdo	6.21	5.92	5.97	6.02	6.02
72	WBd	5.53	5.53	5.49	5.44	5.44
75	WBo	5.03	4.98	5.01	4.74	5.03


In [30]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'CD', [59,61,64,68,69,73])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [59,61,64,68,69,73])
tupleList = hattrickPlayerPostgresql.select(conn, ht_player_table_name, folder, [59,61,64,68,69,73], 'cd')
print('Num', 'b_p', 'b_p_v', 'cd_p', 'cdtw_p', 'cdo_p', sep='\t')
hattrickPlayerPostgresql.print(tupleList, sep='\t')

date num min/rt |    59   |    61   |    64   |    68   |    69   |    73   |
----------------|---------|---------|---------|---------|---------|---------|
2018-10-24      |         |         |  90/5.0 |  90/4.0 |  90/2.5 |         |
2018-10-28      |  90/4.5 |  90/5.0 |         |         |         |  90/2.5 |
2018-10-31      |         |         |  90/5.0 |  90/4.0 |  90/2.5 |         |
2018-11-04      |  90/4.5 |  90/5.0 |         |         |         |  90/3.0 |
2018-11-07      |         |         |  90/5.5 |  90/4.5 |  90/3.0 |         |
2018-11-11      |  90/5.0 |  90/5.0 |         |         |         |  90/3.0 |
2018-11-14      |         |         |  90/5.0 |  90/4.5 |  90/3.0 |         |
2018-11-18      |  90/5.5 |  90/5.0 |         |         |         |  90/3.0 |
2018-11-21      |         |         |  90/5.0 |  90/4.5 |  90/3.0 |         |
Num	b_p	b_p_v	cd_p	cdtw_p	cdo_p
59	CDtw	8.88	8.74	8.88	7.82
61	CDtw	9.03	9.02	9.03	8.67
64	CDtw	8.55	8.37	8.55	7.39
68	CDtw	8.39	8.38	8.39	8.0

In [31]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'W', [7,13,15,43,63])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [7,13,15,43,63])
tupleList = hattrickPlayerPostgresql.select(conn, ht_player_table_name, folder, [7,13,15,43,63], 'w')
print('Num', 'b_p', 'b_p_v', 'wd_p', 'w_p', 'wtm_p, wo_p', sep='\t')
hattrickPlayerPostgresql.print(tupleList, sep='\t')

date num min/rt |    07   |    13   |    15   |    43   |    63   |
----------------|---------|---------|---------|---------|---------|
2018-10-24      |  55/4.0 |         |         |  35/4.5 |  90/6.0 |
2018-10-28      |         |  90/6.5 |  90/7.0 |         |         |
2018-10-31      |  50/4.0 |         |         |  40/4.0 |  90/6.0 |
2018-11-04      |         |  90/6.5 |  90/7.0 |         |         |
2018-11-07      |  50/4.5 |         |         |  40/5.0 |  90/5.5 |
2018-11-11      |         |  90/6.5 |  90/6.5 |         |         |
2018-11-14      |  35/4.5 |         |         |  55/5.0 |  90/5.5 |
2018-11-18      |         |  90/6.5 |  90/6.5 |         |         |
2018-11-21      |  74/3.5 |         |         |  90/5.5 |  16/6.5 |
Num	b_p	b_p_v	wd_p	w_p	wtm_p, wo_p
7	WBd	7.08	6.22	5.84	5.80	5.75
13	WBd	10.55	9.14	8.55	8.50	8.37
15	WBd	9.93	8.74	8.22	8.13	8.17
43	WBd	9.84	7.52	6.53	6.44	6.25
63	IMd	8.8	8.27	8.46	8.66	8.21


In [32]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'IM', [9,10,11,12,31])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [9,10,11,12,31])
tupleList = hattrickPlayerPostgresql.select(conn, ht_player_table_name, folder, [9,10,11,12,31], 'im')
print('Num', 'b_p', 'b_p_v', 'imd_p', 'im_p', 'imtw_p', 'imo_p', sep='\t')
hattrickPlayerPostgresql.print(tupleList, sep='\t')

date num min/rt |    09   |    10   |    11   |    12   |    31   |
----------------|---------|---------|---------|---------|---------|
2018-10-24      |         |  46/5.5 |  44/5.0 |         |  90/6.5 |
2018-10-28      |  90/6.0 |  50/6.0 |         |  40/5.0 |         |
2018-10-31      |         |  90/5.5 |  79/4.5 |         |  11/7.0 |
2018-11-04      |  90/5.5 |  50/6.0 |         |  40/5.0 |         |
2018-11-07      |         |  46/5.5 |         |  90/5.0 |  44/7.0 |
2018-11-11      |  90/5.5 |  50/5.5 |         |  40/5.0 |         |
2018-11-14      |         |  25/5.5 |         |  65/5.5 |  90/6.5 |
2018-11-18      |  90/6.0 |  50/6.0 |         |  40/5.5 |         |
2018-11-21      |         |  25/6.0 |         |  65/5.5 |  90/6.5 |
Num	b_p	b_p_v	imd_p	im_p	imtw_p	imo_p
9	CD	10.68	8.84	8.53	8.41	8.20
10	CD	9.89	8.75	8.40	8.16	8.01
11	CD	7.78	6.96	6.79	6.66	6.60
12	IMd	8.5	8.50	8.48	8.26	8.44
31	CD	10.03	9.88	9.78	9.26	9.65


In [33]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'FW', [16,18,60])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [16,18,60])
# 16 fw_w
# 60 fw
print('num','b_p', 'b_p_v', 'fw_p', 'fwd_p', 'fwtw_p', 'tdf_p', sep='\t')
tupleList = hattrickPlayerPostgresql.select(conn, ht_player_table_name, folder, [16,18,60], 'fw')
hattrickPlayerPostgresql.print(tupleList, sep='\t')

date num min/rt |    16   |    18   |    60   |
----------------|---------|---------|---------|
2018-10-24      |         |  80/5.5 |  10/5.0 |
2018-10-28      |  81/6.0 |         |   9/5.0 |
2018-10-31      |         |  80/5.5 |  10/5.0 |
2018-11-04      |  35/6.0 |         |         |
2018-11-07      |         |  80/5.5 |  10/5.0 |
2018-11-11      |         |  65/5.5 |  25/5.0 |
2018-11-14      |  90/6.0 |         |         |
2018-11-18      |         |  70/5.5 |  20/5.0 |
2018-11-21      |  90/6.0 |         |         |
num	b_p	b_p_v	fw_p	fwd_p	fwtw_p	tdf_p
16	FWtw	7.3	7.11	6.36	7.30	0.00
18	FWd	6.46	6.42	6.46	6.20	0.00
60	FW	5.94	5.94	5.45	5.74	0.00


In [34]:
conn.close()
print(conn)

<connection object at 0x7ff7e2d5b768; dsn: 'user=myuser password=xxx dbname=mydatabase2 host=localhost port=65432', closed: 1>


In [35]:
conn = None
print(conn)

None
